# SIXT33N Project
## Phase 4: Advanced Controls (and more!)

### EE16B: Designing Information Devices and Systems II, Fall 2019

Written by Vivek Athalye and Andrew Blatner (2017). Revised by Peter Schafhalter (2019).

## Table of Contents

* [Introduction](#intro)
* [Part 1: Analyzing a Turn](#part1)
* [Part 2: Implementing Turns](#part2)
* [Part 3: Correcting Mechanical Errors](#part3)
* [Part 4: Launchpad Implementation of PCA Classify (Optional)](#part4)

<a id='intro'></a>
## Introduction

This is the final lab for those who don't plan on doing the optional PCA classification. Recall that our control scheme for SIXT33N modeled each wheel in the following way:

$$d_R[k+1] = d_R[k] + \theta_R u_R[k] - \beta_R$$

The open loop model has an input $u[k]$, corresponding to the PWM level, and outputs the distance for that wheel. We implemented a closed-loop controller by using the desired velocity and the difference between the wheel distances, $$\delta[k] = d_L[k] - d_R[k]$$.

In this phase, we will implement turns by modifying the closed loop control scheme we developed to make SIXT33N move straight.

<a id='part1'></a>
## <span style="color:navy">Part 1: Analyzing a Turn</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme

First, verify that your closed loop control scheme is still functional. Load `closed_loop.ino` onto your launchpad and make sure your car drives straight.

### We want to turn with a specific radius.  What open-loop control input would lead to turning?  
During open-loop control, we sought to drive the car straight at our desired velocity $v^*$.  

We used our model of the car ($\theta$, $\beta$) to calculate the open-loop control input which drives the car straight in the absence of perturbations:  

$$u^{OL}_L = \frac{v^*+\beta_L}{\theta} $$
$$u^{OL}_R = \frac{v^*+\beta_R}{\theta} $$

Is it possible to calculate an open-loop control input which would result in turning with a desired radius?


### We have been turning during Closed-loop Control
We can get a clue about turning by considering how we did closed-loop control. We perform feedback control of SIXT33N in order to minimize the difference between the two wheels ($\delta[k] = d_L[k] - d_R[k]$). When perturbations cause one wheel to get ahead of the other resulting in non-zero $\delta$, feedback control turns the car to correct the error.  

Let's walk through this.  

Our feedback control policy is: 
$$u_L[k] = u_L^{OL} - \frac{k_L}{\theta_L}\delta[k]$$

$$u_R[k] = u_R^{OL} + \frac{k_R}{\theta_R}\delta[k]$$


Let's say the right wheel has moved further than the left ($d_R[k] > d_L[k]$), resulting in a negative $\delta[k]$. A negative $\delta[k]$ results in a reduction of $u_R[k]$, and an increase of $u_L[k]$. But what does this mean physically?

If the right wheel is getting ahead of the left one, the left wheel will need to move a little faster and the right wheel a little slower for the wheels to even back out. Thus, feedback control corrects errors by turning to compensate for them.  

So $\delta \ne 0$ can be used to turn the car. Let's get more precise.  


### Turning via reference tracking

We would like the car to turn with a specified radius $r$ and speed $v^*$. The controller's unit for distance is encoder ticks, but each tick is approximately 1 cm of wheel circumference.

To turn, we want $\delta$ to change at a particular rate. Without loss of generality, we'll analyze a right turn, corresponding to increasing $\delta$. For a left turn, we simply negate $\delta$. Our goal is to generate a reference from the desired $r$ and $v^*$ for the controller to follow. This reference will be a function of the controller's time-step.

Use the following variables:
- $k$ [time] - time-step
- $r$ [cm] - turn radius of the center of the car; 1 cm ~ 1 encoder tick
- $d$ [tick] - distance traveled by the center of the car
- $l$ [cm] - distance between the centers of the wheels; 1 cm ~ 1 encoder tick
- $\omega$ [rad/time] - angular velocity
- $\theta$ [rad] - angle traveled

Inspect the following diagram:

<center>
<img width="400px" src="images/turning_figure.png">
</center>



From this geometry, can you write $\delta[k]$ in the following form?
$$\delta[k] = f(r,v^*,l,k)$$

<h3><span style='color:red'>**Check your answer with the lab staff!**</span></h3>

Answer:

<a id='part2'></a>
## <span style="color:navy">Part 2: Implementing Turns</span>

### Materials
- SIXT33N Car
- Closed-loop control scheme
- Plan from Task 1

### Sampling periods 
 
In our implementation, the control loop and the data collection have different periods, $T_c$ and $T_d$ respectively, and therefore different sampling frequencies, $F_c$ and $F_d$, respectively.  $F_c$ is an integer multiple of $F_d$: $m=\frac{F_c}{F_d} \in \mathbb{Z}^{+}$

To ensure the car is travelling at the same ground velocity in both cases, we want to have a constant distance, measured in ticks, travelled per second, or $\frac{ticks}{s}$.

$F_c$, with units [$\frac{sample_c}{s}$], is the number of times the closed loop is sampled every second, and $F_d$, with units with units [$\frac{sample_d}{s}$], is the number of times data collection was sampled per second. Let $v^*_c$ be the velocity of closed loop, and $v^*$ be the velocity of the data collection. $v^*_c$ has the units of [$\frac{ticks}{sample_c}$] and $v^*$ has units [$\frac{ticks}{sample_d}$].

To get a constant $\frac{ticks}{s}$, we need $v^*_c * F_c = v^* * F_d$.

Solving for $v^*_c$, we get $v^*_c = v^* * \frac{F_d}{F_c} = \frac{v^*}{m}$

We use $T_c=100$ms, $T_d=500ms$, so $F_c=10$, $F_d=2$, so $m=5$.

**<span style='color:red'> Replace $v^*$ in your code with $\frac{v^*}{m}$</span>**

### Implementation
- Open `turning.ino`.
- Copy the required code from `closed_loop.ino` into **`CODE BLOCK CON1/2/3`**
    - Make sure to update pin numbers if you've made changes.
    - Setting `delta_ss` helps your car's driving converge to straight sooner!
    - Note that your `driveStraight` only have one parameter in this program unlike closed_loop.ino, so only copy over the body of the function!
- **Complete the function `delta_reference` in `CODE BLOCK CON4`**.
    - Look at `CODE BLOCK CON0` to see how this is used!
    - Use the variables `TURN_RADIUS`, `v_star`, `CAR WIDTH`, and `k`.
    - `delta_ss` is used in `CODE BLOCK CON0`, so don't include it here.
- Program a command string for your car to follow in **`PREPROGRAMMED PATH`**
    - Set `NUM_COMMANDS` to the number of commands you want your car to follow.
    - In the `run_times` and `drive_modes` arrays, add at least 8 commands you want your car to follow as well as the length associated with each command.
        - The number of elements in both `drive_modes` and `run_times` should be equal to `NUM_COMMANDS`
        - The values in `run_times` are (roughly) in ms
        - `drive_modes` can take on values: `[DRIVE_STRAIGHT, DRIVE_LEFT, DRIVE_RIGHT]`. Make sure to include at least one of each command in your path!

When you're done, try it out! SIXT33N should follow your preprogrammed commands with a stop between each stage. It's ok if SIXT33N does not turn 90 degrees - as long as it turns you should be fine. If you want to adjust the turn you can change the turn radius or the turn's runtime until it only turns 90 degrees. Just make sure that it is clear when SIXT33N is turning, and when it is driving straight.

<a id='part3'></a>
## <span style="color:navy">Part 3: Correcting Mechanical Errors</span>

At this stage, your car may already go perfectly straight and turn left and right equally. However, some cars may not go quite straight even when both encoder measurements are exactly the same. This can be caused by mechanical issues such as axle wobble or mismatch in the wheel sizes. In this case, the car thinks it's driving straight, so it's the best we can do using just the sensors in our control system.

If the car is consistently turning slightly instead of driving straight, we can correct it by instructing the car to slightly turn in the opposite direction. 

**If you think your car could benefit from this, implement the function `straight_correction` in `CODE BLOCK CON5` using the variable `STRAIGHT_RADIUS`.**

<a id='part4'></a>
## <span style="color:navy">Part 4: Launchpad Implementation of PCA Classify (Optional)</span>
### NOTE: If you're not implementing voice control, skip to the [end](#Write-up).

### Materials
- Microphone front-end circuit
- Launchpad + USB


**<span style="color:red">IMPORTANT: remove the 5V power pin from the launchpad for this step. It can cause interference during classification.</span>**

This section will walk you through implementing your classification algorithm on the Launchpad. In addition, you will need to transfer the PCA vectors and mean to the Launchpad. **Copy/paste the code from the Appendix of the PCA notebook into `classify.ino.`**

Your last step will be to implement your <b>data processing</b> and <b>classification</b> (just the projection, not the PCA) in the Launchpad sketch <b>`classify.ino`</b>. Since Energia does not have as many in-built functions as Python, you might have to write out the functions yourself. For example, a dot product should be written as:

```C
float result = 0;
for (i = 0; i < LENGTH; i++) {
    result += vector1[i] * vector2[i];
}
```

**HINT:** how could you modify the code above to project `vector1` onto `vector2` **and** `vector3` using only 1 loop?

For debugging purposes, printing to Energia's serial monitor looks like the line below.

`Serial.println("I'm being printed!");`

There are 3 code blocks (`PCA1/2/3`) that you need to modify. <b>You should not have to change anything else outside these marked code blocks.</b> 

**`CODE BLOCK PCA1`**
- Copy `SNIPPET_SIZE`, `PRELENGTH` and `THRESHOLD` from the PCA notebook.
- Read the following to set `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD`.

**`KMEANS_THRESHOLD` filters the classification depending on a sample's distance from the closest centroid.** If the L2 norm (distance) is larger than the threshold, your classification algorithm should simply ignore it and wait for the next sample. Look at the plot of sample data and the centroids from the PCA Notebook and approximate a radius around the centroids that capture most of the data. **Try to be conservative - it's better to not classify than to misclassify.**

**`LOUDNESS_THRESHOLD` filters the classification depending on the amplitude of the recorded data.** If the recorded data is too soft, we do not want to classify it as it is probably noise. Since the loudness unit is arbitrary, start by using `700`. Later, if the Launchpad classifies noise, increase this constant. If it misses a lot of speech (i.e. thinks your word is noise), decrease this constant. This variable is used internally in the enveloping function.

**`CODE BLOCK PCA2`**
- Copy the PCA vectors, mean, and centroids from the PCA notebook.
- If you need more than 2 principal components, add a new `pca_vec3` array. 
- Using more principal components increases the dimensionality of the centroids and projections.

**`CODE BLOCK PCA3`**
- This is the actual classification algorithm.
- Before this block, the call to `envelope` leaves the data vector in the array called `result`.
- Project this data onto your new PCA basis.
    - Use the variables `proj1` and `proj2` to store the projection results.
- Demean the projection.
    - Remember that we *demean after projecting* to save memory on the launchpad. Instead of $ y = P(x - \bar{x}) $, we run $ y = Px - \bar{x}_{\text{proj}} $ where $ \bar{x}_{\text{proj}} = P \bar{x} $.
- Classify the projections using the centroids.
    - Find the distance between the projected data point and each centroid using the function `l2_norm` (for 2 principal components) or `l2_norm3` (for 3 principal components). Look up the function definition in the sketch.
    - Out of the 4 centroids, find the one with the smallest L2 norm.
    - Verify this distance is less than `KMEANS_THRESHOLD`.
- Print the classification to the serial monitor.

Before testing the code, probe the circuit's output with the oscilloscope and make sure that it still averages around 1.65V. Now upload the sketch, (re)open the serial monitor, and press the reset button. Say your word and the Launchpad should recognize it!

**<span style="color:red">If the Launchpad does not classify as well as you think it should, remember to play with the `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD` variables.</span>** To debug the sketch, you can also print out any of the variables you have used. 

Voila! Your SIXT33N can recognize your words!

**<span style="color:red">Summary TO DO</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA1`: Fill out `SNIPPET_SIZE`, `PRELENGTH`, `THRESHOLD`, `KMEANS_THRESHOLD` and `LOUDNESS_THRESHOLD`</span>** 
- **<span style="color:red">Fill in `CODE BLOCK PCA2`: Copy the principal components, mean vector and centroids from the IPython notebook</span>**
- **<span style="color:red">Fill in `CODE BLOCK PCA3`: Do the actual classification.</span>**

<a id='Write-up'></a>
## <span style="color:blue">Final Demo and Write-up</span>
Congratulations - You're done with lab! If you have some time, be creative and try to add a functionality to the SIXT33N! Don't hesitate to ask for your GSI's help when you want to modify the Launchpad sketch since it is more complicated than previous sketches. 

As a side note, the EECS 16B lab staff encourages you to look through (and/or complete) the optional voice commands part of the lab (although it is not necessary for full credit). The **SVD/PCA** lab teaches you a lot about signal processing, audio circuitry, and some very rudimentary machine learning that will broaden your horizons as an electrical engineering student. If you do want to complete these labs, and want to talk to a GSI about any questions you may have feel free to reach out to any of us!

### Demo

You have the option to either demo the project during your lab time or to take a video of it working properly. In the demo, you will have to:
- Set SIXT33N on the ground, then let it run its preprogrammed instructions.
- You are allowed to reposition the car inbetween the steps if you have trouble crashing into walls.


### Write-up

On top of the demo, you will need to submit a write-up of the project. Submit the write-up as a PDF to Gradescope by **TBA**. We will release more details about this and a rubric soon.
